In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [2]:
from pathlib import Path

import pandas as pd
import tensorflow as tf
from PIL import Image
from tensorflow import keras

## Build a universal one hot encoder that encodes cross-dataset category and ingredients 

In [3]:
class OneHotEncoder:
    def __init__(self, all_category_list, all_ingredient_list):
        self.all_food_categories = all_category_list
        self.all_food_categories.sort()
        self.all_food_categories_integer_encoded = (
            self.__encode_categories_to_integers()
        )
        self.all_ingredients = all_ingredient_list
        self.all_ingredients.sort()
        self.all_ingredients_integer_encoded = self.__encode_ingredients_to_integers()

    def get_category_one_hot_encoding(self, category_name):
        index = self.all_food_categories_integer_encoded[category_name]
        assert index != None, f"{category_name} does not have an integer mapping"
        num_classes = len(self.all_food_categories)
        return keras.utils.to_categorical(index, num_classes)

    def get_ingredients_one_hot_encoding(self, ingredient_list):
        ingredient_list = list(
            map(lambda x: self.__transform_ingredient_to_integer(x), ingredient_list)
        )
        multi_one_hot_layer = tf.keras.layers.CategoryEncoding(
            num_tokens=len(self.all_ingredients), output_mode="multi_hot"
        )
        return multi_one_hot_layer(ingredient_list)

    def __transform_ingredient_to_integer(self, ingredient_name):
        index = self.all_ingredients_integer_encoded[ingredient_name]
        assert index != None, f"{ingredient_name} does not have an integer mapping"
        return index

    def __encode_categories_to_integers(self):
        return {
            category_name: index
            for index, category_name in enumerate(self.all_food_categories)
        }

    def __encode_ingredients_to_integers(self):
        return {
            ingredient_name: index
            for index, ingredient_name in enumerate(self.all_ingredients)
        }

## Build dataset loaders for each dataset

In [4]:
class DatasetLoader:
    def __init__(self, image_dir, metadata_dir, dataset_name):
        self.image_dir = Path(image_dir)
        self.metadata_dir = Path(metadata_dir)
        self.name = dataset_name
        self.metadata = self.load_metadata(
            self.metadata_dir / ("{dataset}_metadata.csv".format(dataset=dataset_name))
        )
        # Default : all_files = metadata, since metadata consists of records of all files
        self.all_files = self.metadata.copy()
        self.all_categories = self.extract_all_categories()
        self.all_ingredients = self.extract_all_ingredients()

    def load_image_to_arr(self, path):
        image = tf.keras.preprocessing.image.load_img(path)
        img_tensor = tf.keras.preprocessing.image.img_to_array(image)
        return tf.image.resize(img_tensor, (224, 224))

    def load_metadata(self, path):
        metadata = pd.read_csv(path, sep="\t").sample(frac=1, random_state=1234)
        new_metadata = metadata.copy()
        new_metadata["dataset_name"] = self.name
        return new_metadata

    def extract_all_categories(self):
        return self.metadata["Category"].unique().tolist()

    def extract_all_ingredients(self):
        unique_ingredients = set()
        for ingredient_list in self.metadata["Ingredients"]:
            ingredient_list = ingredient_list.split(",")
            unique_ingredients.update(ingredient_list)
        return [*unique_ingredients]

    def extract_file_pointers(self):
        dataset_name_col = self.all_files["dataset_name"]
        index_col = self.all_files.index
        return pd.DataFrame(
            {"metadata_index": index_col, "dataset_name": dataset_name_col}
        )

    def get_tensors(self, index, one_hot_encoder):
        img_dir = self.image_dir
        row = self.all_files.loc[index]
        img_path = img_dir / row["Category"] / row["ID/File Name"]
        img_tensor = self.load_image_to_arr(img_path)
        calorie_tensor = row["Calorie(kcal)"]
        carbs_tensor = row["Carbohydrate(g)"]
        protein_tensor = row["Protein(g)"]
        fat_tensor = row["Fat(g)"]
        one_hot_category_tensor = one_hot_encoder.get_category_one_hot_encoding(
            row["Category"]
        )
        one_hot_ingredient_tensor = one_hot_encoder.get_ingredients_one_hot_encoding(
            row["Ingredients"].split(",")
        )
        return tf.constant(img_tensor), {
            "category_output": tf.constant(one_hot_category_tensor),
            "calorie_output": tf.constant(calorie_tensor),
            "carbs_output": tf.constant(carbs_tensor),
            "protein_output": tf.constant(protein_tensor),
            "fat_output": tf.constant(fat_tensor),
            "ingredients_output": one_hot_ingredient_tensor,
        }

    def flatten_tensors(self, tensor):
        result = []
        img_data = tensor[0].numpy()
        others_data = [value.numpy() for key, value in tensor[1].items()]
        metadata = [key for key in tensor[1].keys()]
        result.append(img_data)
        result.extend(others_data)
        return result

    def __len__(self):
        return len(self.metadata)

In [5]:
class Recipes5k(DatasetLoader):
    def __init__(self, image_dir, metadata_dir):
        super().__init__(image_dir, metadata_dir, "recipes5k")

In [6]:
class Nutrition5k(DatasetLoader):
    def __init__(self, image_dir, metadata_dir):
        super().__init__(image_dir, metadata_dir, "nutrition5k")
        # Modify all_files since nutrition5k metadata only consists dish_level metadata not image_level
        self.all_files = pd.read_csv(self.metadata_dir / "nutrition5k_all_images.csv")

    # Override method from DatasetLoader
    def get_tensors(self, index, one_hot_encoder):
        img_dir = self.image_dir
        row = self.all_files.loc[index]
        img_path = img_dir / "generic" / row["dish_id"] / row["ID/File Name"]
        img_tensor = self.load_image_to_arr(img_path)
        dish_metadata_row = self.metadata.loc[
            self.metadata["dish_id"] == row["dish_id"]
        ].squeeze()
        calorie_tensor = dish_metadata_row["Calorie(kcal)"]
        carbs_tensor = dish_metadata_row["Carbohydrate(g)"]
        protein_tensor = dish_metadata_row["Protein(g)"]
        fat_tensor = dish_metadata_row["Fat(g)"]
        one_hot_category_tensor = one_hot_encoder.get_category_one_hot_encoding(
            dish_metadata_row["Category"]
        )
        one_hot_ingredient_tensor = one_hot_encoder.get_ingredients_one_hot_encoding(
            dish_metadata_row["Ingredients"].split(",")
        )
        return tf.constant(img_tensor), {
            "category_output": tf.constant(one_hot_category_tensor),
            "calorie_output": tf.constant(calorie_tensor),
            "carbs_output": tf.constant(carbs_tensor),
            "protein_output": tf.constant(protein_tensor),
            "fat_output": tf.constant(fat_tensor),
            "ingredients_output": one_hot_ingredient_tensor,
        }

    # Overrding the method from DatasetLoader
    def __len__(self):
        return len(self.all_files)

In [7]:
class Food101(DatasetLoader):
    def __init__(self, image_dir, metadata_dir):
        super().__init__(image_dir, metadata_dir, "food101")

## Initializing one hot encoder

In [8]:
# Get all the categories and ingredients from all datasets

# Initialize dataset loader without one-hot encoder to get all unique category and ingredients from each dataset
recipes5k = Recipes5k(
    image_dir="../input/foodnet/images/images",
    metadata_dir="../input/foodnet/metadata",
)
nutrition5k = Nutrition5k(
    image_dir="../input/foodnet/images/images",
    metadata_dir="../input/foodnet/metadata",
)
food101 = Food101(
    image_dir="../input/foodnet/images/images",
    metadata_dir="../input/foodnet/metadata",
)

all_categories = recipes5k.all_categories
all_categories.extend(nutrition5k.all_categories)
all_categories.extend(food101.all_categories)
all_categories = set(all_categories)

all_ingredients = recipes5k.all_ingredients
all_ingredients.extend(nutrition5k.all_ingredients)
all_ingredients.extend(food101.all_ingredients)
all_ingredients = set(all_ingredients)

In [9]:
one_hot_encoder = OneHotEncoder([*all_categories], [*all_ingredients])

## Building dataset pipeline

In [10]:
datasets = [recipes5k, nutrition5k, food101]
dataset_name = [x.name for x in datasets]


def get_dataset_cardinality(datasets):
    dataset_samples = [len(x) for x in datasets]
    return sum(dataset_samples)


def get_file_data(index, dataset_index):
    index = index.numpy()
    dataset_index = dataset_index.numpy()
    target_dataset = datasets[dataset_index]
    return target_dataset.flatten_tensors(
        target_dataset.get_tensors(index, one_hot_encoder)
    )


def transform_file_pointers(index, dataset_index):
    result = tf.py_function(
        get_file_data,
        [index, dataset_index],
        [
            tf.float32,
            tf.float32,
            tf.float32,
            tf.float32,
            tf.float32,
            tf.float32,
            tf.float32,
        ],
    )
    return result[0], {
        "category_output": result[1],
        "calorie_output": result[2],
        "carbs_output": result[3],
        "protein_output": result[4],
        "fat_output": result[5],
        "ingredients_output": result[6],
    }


def build_data_pipeline(datasets, sample_size=None):
    if sample_size == None:
        sample_size = [1.0] * len(datasets)
    assert len(sample_size) == len(
        datasets
    ), "Illegal array of sample sizes provided. Number of sample size does not match number of datasets"
    file_pointers = [
        x.extract_file_pointers().sample(frac=s) for x, s in zip(datasets, sample_size)
    ]
    all_file_pointers = pd.concat(file_pointers)
    print(f"Total samples : {len(all_file_pointers)}")
    all_file_pointers["dataset_name"] = all_file_pointers["dataset_name"].apply(
        lambda x: dataset_name.index(x)
    )
    all_file_pointers = [
        (index, dataset_name)
        for index, dataset_name in all_file_pointers.itertuples(index=False)
    ]
    final_dataset = tf.data.Dataset.from_generator(
        lambda: all_file_pointers,
        output_signature=(
            tf.TensorSpec(shape=(), dtype=tf.int32),
            tf.TensorSpec(shape=(), dtype=tf.int32),
        ),
    )
    final_dataset = final_dataset.shuffle(get_dataset_cardinality(datasets))
    final_dataset = final_dataset.map(
        lambda index, name: transform_file_pointers(index, name),
        num_parallel_calls=tf.data.AUTOTUNE,
    )
    return final_dataset


def split_dataset(dataset, total_samples, training_split, batch_size):
    training_size = int(total_samples * training_split)
    training_dataset = (
        dataset.take(training_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    )
    validation_dataset = (
        dataset.skip(training_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    )
    print(f"Training size : {training_size}")
    print(f"Validation size : {total_samples-training_size}")
    return training_dataset, validation_dataset

In [11]:
final_dataset = build_data_pipeline(
    [recipes5k, food101]
)

Total samples : 105826


In [ ]:
list(final_dataset.take(3))

In [12]:
training_dataset, validation_dataset = split_dataset(final_dataset, 105826, 0.7, 32)

Training size : 74078
Validation size : 31748


## MobileNetv2 Convolution Base Model Building

In [13]:
class BaseModel:
    def __init__(
        self, input_shape, total_food_category, total_ingredients_category, name
    ):
        self.input_shape = input_shape
        self.input_layer = self.get_input_layer()
        self.model = None
        self.name = name

    def get_input_layer(self):
        return keras.Input(shape=self.input_shape)

    def save_model(self, path):
        assert self.model != None
        self.model.save(path, save_format="h5")

    def load_model(self, path):
        self.model = keras.models.load_model(path)

In [14]:
class FlatModel(BaseModel):
    def __init__(
        self, input_shape, total_food_category, total_ingredients_category, name
    ):
        super().__init__(
            input_shape, total_food_category, total_ingredients_category, name
        )
        self.augmentation_layers = self.get_augmentation_layers()
        self.preprocess_layers = self.get_preprocess_layers(self.augmentation_layers)
        self.convolution_block = self.get_convolution_block()
        self.shared_layers = self.get_shared_layers(self.convolution_block)

        self.category_classification_layers = self.get_category_classification_layers(
            self.shared_layers, total_food_category
        )
        self.shared_nutrition_regression_layers = self.get_shared_nutrition_layers(
            self.shared_layers
        )
        self.calorie_regression_layers = self.get_calorie_regression_layers(
            self.shared_nutrition_regression_layers
        )
        self.carbs_regression_layers = self.get_carbs_regression_layers(
            self.shared_nutrition_regression_layers
        )
        self.protein_regression_layers = self.get_protein_regression_layers(
            self.shared_nutrition_regression_layers
        )
        self.fat_regression_layers = self.get_fat_regression_layers(
            self.shared_nutrition_regression_layers
        )
        self.ingredients_multilabel_layers = self.get_ingredients_multilabel_layers(
            self.shared_layers, total_ingredients_category
        )

    def get_augmentation_layers(self):
        input_layer = keras.layers.Input(shape=self.input_shape)
        augmentation_layer = keras.layers.RandomFlip()(input_layer)
        augmentation_layer = keras.layers.RandomRotation(0.2)(augmentation_layer)
        return keras.Model(
            inputs=input_layer, outputs=augmentation_layer, name="augmentation_layers"
        )

    def get_preprocess_layers(self, previous_layer):
        return None

    def get_convolution_block(self):
        return None

    def get_shared_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        shared_layer = keras.layers.Flatten()(input_layer)
        shared_layer = keras.layers.Dense(
            128, activation="relu", name="shared_dense_1"
        )(shared_layer)
        shared_layer = keras.layers.BatchNormalization()(shared_layer)
        output_layer = keras.layers.Dropout(0.2)(shared_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="shared_layers"
        )

    def get_category_classification_layers(self, previous_layer, total_categories):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        category_classification_layer = keras.layers.Dense(
            total_categories, activation="softmax", name="category_output"
        )(input_layer)
        return keras.Model(
            inputs=input_layer,
            outputs=category_classification_layer,
            name="category_output",
        )

    def get_shared_nutrition_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        shared_nutrition_regression_layers = keras.layers.Dense(
            128, activation="relu", name="nutrition_dense_1"
        )(input_layer)
        shared_nutrition_regression_layers = keras.layers.BatchNormalization()(
            shared_nutrition_regression_layers
        )
        output_layer = keras.layers.Dropout(0.2)(shared_nutrition_regression_layers)

        return keras.Model(
            inputs=input_layer,
            outputs=output_layer,
            name="nutrition_regression_shared_layers",
        )

    def get_calorie_regression_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        calorie_regression_layers = keras.layers.Dense(1, name="calorie_output")(
            input_layer
        )
        return keras.Model(
            inputs=input_layer, outputs=calorie_regression_layers, name="calorie_output"
        )

    def get_carbs_regression_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        carbs_regression_layers = keras.layers.Dense(1, name="carbs_output")(
            input_layer
        )
        return keras.Model(
            inputs=input_layer, outputs=carbs_regression_layers, name="carbs_output"
        )

    def get_protein_regression_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        protein_regression_layers = keras.layers.Dense(1, name="protein_output")(
            input_layer
        )
        return keras.Model(
            inputs=input_layer, outputs=protein_regression_layers, name="protein_output"
        )

    def get_fat_regression_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        fat_regression_layers = keras.layers.Dense(1, name="fat_output")(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=fat_regression_layers, name="fat_output"
        )

    def get_ingredients_multilabel_layers(self, previous_layer, total_ingredients):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        ingredients_multilabel_layers = []
        ingredients_multilabel_layers = keras.layers.Dense(
            128, activation="relu", name="ingredients_dense_1"
        )(input_layer)
        ingredients_multilabel_layers = keras.layers.BatchNormalization()(
            ingredients_multilabel_layers
        )
        ingredients_multilabel_layers = keras.layers.Dropout(0.2)(
            ingredients_multilabel_layers
        )
        output_layers = keras.layers.Dense(
            total_ingredients, activation="sigmoid", name="ingredients_output"
        )(ingredients_multilabel_layers)
        return keras.Model(
            inputs=input_layer, outputs=output_layers, name="ingredients_output"
        )

    def build_and_compile(
        self,
        category_classification_loss=keras.losses.CategoricalCrossentropy(),
        calorie_regression_loss=keras.losses.MeanAbsoluteError(),
        carbs_regression_loss=keras.losses.MeanAbsoluteError(),
        protein_regression_loss=keras.losses.MeanAbsoluteError(),
        fat_regression_loss=keras.losses.MeanAbsoluteError(),
        ingredient_multilabel_loss=keras.losses.BinaryCrossentropy(),
        category_classification_metrics=[
            keras.metrics.CategoricalAccuracy(),
            keras.metrics.Precision(),
            keras.metrics.Recall(),
        ],
        calorie_regression_metrics=[keras.metrics.MeanAbsoluteError()],
        carbs_regression_metrics=[keras.metrics.MeanAbsoluteError()],
        protein_regression_metrics=[keras.metrics.MeanAbsoluteError()],
        fat_regression_metrics=[keras.metrics.MeanAbsoluteError()],
        ingredient_multilabel_metrics=[
            keras.metrics.TopKCategoricalAccuracy(5),
            keras.metrics.Precision(),
            keras.metrics.Recall(),
        ],
        category_classification_loss_weights=1.0,
        ingredient_multilabel_loss_weights=1.0,
        calorie_regression_loss_weights=1.0,
        carbs_regression_loss_weights=1.0,
        protein_regression_loss_weights=1.0,
        fat_regression_loss_weights=1.0,
    ):
        assert (
            self.preprocess_layers != None
        ), "Error: No preprocess layers exists. Class FlatModel should not be instantiated."
        assert (
            self.convolution_block != None
        ), "Error: No convolution block exists. Class FlatModel should not be instantiated."
        model_inputs = self.input_layer
        x = self.augmentation_layers(model_inputs)
        x = self.preprocess_layers(x)
        x = self.convolution_block(x)
        x = self.shared_layers(x)
        category_classification_head = self.category_classification_layers(x)
        nutrition_regression_head = self.shared_nutrition_regression_layers(x)
        ingredients_multilabel_head = self.ingredients_multilabel_layers(x)
        calorie_regression_head = self.calorie_regression_layers(
            nutrition_regression_head
        )
        carbs_regression_head = self.carbs_regression_layers(nutrition_regression_head)
        protein_regression_head = self.protein_regression_layers(
            nutrition_regression_head
        )
        fat_regression_head = self.fat_regression_layers(nutrition_regression_head)

        model = keras.Model(
            inputs=model_inputs,
            outputs=[
                category_classification_head,
                ingredients_multilabel_head,
                calorie_regression_head,
                carbs_regression_head,
                protein_regression_head,
                fat_regression_head,
            ],
            name=self.name,
        )
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss={
                "category_output": category_classification_loss,
                "calorie_output": calorie_regression_loss,
                "carbs_output": carbs_regression_loss,
                "protein_output": protein_regression_loss,
                "fat_output": fat_regression_loss,
                "ingredients_output": ingredient_multilabel_loss,
            },
            metrics={
                "category_output": category_classification_metrics,
                "calorie_output": calorie_regression_metrics,
                "carbs_output": carbs_regression_metrics,
                "protein_output": protein_regression_metrics,
                "fat_output": fat_regression_metrics,
                "ingredients_output": ingredient_multilabel_metrics,
            },
            loss_weights={
                "category_output": category_classification_loss_weights,
                "calorie_output": calorie_regression_loss_weights,
                "carbs_output": carbs_regression_loss_weights,
                "protein_output": protein_regression_loss_weights,
                "fat_output": fat_regression_loss_weights,
                "ingredients_output": ingredient_multilabel_loss_weights,
            },
        )
        self.model = model
        return model

In [15]:
class FlatMobileNetv2Model(FlatModel):
    def __init__(self, input_shape, total_food_category, total_ingredients_category):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "FlatFoodNet_with_MobileNetv2",
        )
        self.preprocess_layers = self.get_preprocess_layers(self.augmentation_layers)
        self.convolution_block = self.get_convolution_block()

    def get_preprocess_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        output_layer = keras.applications.mobilenet_v2.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        mobilenet_v2_convolution_layers = keras.applications.MobileNetV2(
            input_shape=self.input_shape, include_top=False, weights="imagenet"
        )
        mobilenet_v2_convolution_layers.trainable = False
        return mobilenet_v2_convolution_layers

In [16]:
flat_foodnet_mobilenetv2 = FlatMobileNetv2Model(
    input_shape=(224, 224, 3),
    total_food_category=len(one_hot_encoder.all_food_categories),
    total_ingredients_category=len(one_hot_encoder.all_ingredients),
)
flat_foodnet_mobilenetv2.build_and_compile()
flat_foodnet_mobilenetv2.model.summary()

Model: "FlatFoodNet_with_MobileNetv2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
augmentation_layers (Functional (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
preprocessing_layers (Functiona (None, 224, 224, 3)  0           augmentation_layers[0][0]        
__________________________________________________________________________________________________
mobilenetv2_1.00_224 (Functiona (None, 7, 7, 1280)   2257984     preprocessing_layers[0][0]       
_______________________________________________________________________

In [ ]:
keras.utils.plot_model(
    flat_foodnet_mobilenetv2.model,
    "./models/flat_foodnet_mobilenetv2.png",
    show_shapes=True,
    expand_nested=False,
)

In [17]:
flat_foodnet_mobilenetv2.model.fit(
    training_dataset, epochs=10, verbose=1, validation_data=validation_dataset
)

Epoch 1/10
2315/2315 [==============================] - 1972s 842ms/step - loss: 3.9299 - category_output_loss: 2.6648 - ingredients_output_loss: 0.0677 - calorie_output_loss: 0.7662 - carbs_output_loss: 0.1587 - protein_output_loss: 0.1055 - fat_output_loss: 0.1670 - category_output_categorical_accuracy: 0.3623 - category_output_precision: 0.8018 - category_output_recall: 0.1504 - ingredients_output_top_k_categorical_accuracy: 0.1169 - ingredients_output_precision_1: 0.0606 - ingredients_output_recall_1: 0.1067 - calorie_output_mean_absolute_error: 0.7662 - carbs_output_mean_absolute_error: 0.1587 - protein_output_mean_absolute_error: 0.1055 - fat_output_mean_absolute_error: 0.1670 - val_loss: 2.6899 - val_category_output_loss: 1.8760 - val_ingredients_output_loss: 0.0310 - val_calorie_output_loss: 0.6042 - val_carbs_output_loss: 0.0804 - val_protein_output_loss: 0.0272 - val_fat_output_loss: 0.0711 - val_category_output_categorical_accuracy: 0.5245 - val_category_output_precision: 0.

In [ ]:
class FlatResNet50Model(FlatModel):
    def __init__(self, input_shape, total_food_category, total_ingredients_category):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "FlatFoodNet_with_ResNet50",
        )
        self.preprocess_layers = self.get_preprocess_layers(self.augmentation_layers)
        self.convolution_block = self.get_convolution_block()

    def get_preprocess_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        output_layer = keras.applications.resnet50.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        resnet50_convolution_layers = keras.applications.ResNet50(
            input_shape=self.input_shape, include_top=False, weights="imagenet"
        )
        resnet50_convolution_layers.trainable = False
        return resnet50_convolution_layers

In [ ]:
flat_foodnet_resnet50 = FlatResNet50Model(
    input_shape=(224, 224, 3),
    total_food_category=len(one_hot_encoder.all_food_categories),
    total_ingredients_category=len(one_hot_encoder.all_ingredients),
)
flat_foodnet_resnet50.build_and_compile()
flat_foodnet_resnet50.model.summary()

In [ ]:
flat_foodnet_resnet50.model.fit(
    training_dataset, epochs=1, verbose=1, validation_data=validation_dataset
)

## Region Wise Model

In [ ]:
class RegionWiseModel(BaseModel):
    def __init__(
        self, input_shape, total_food_category, total_ingredients_category, name
    ):
        super().__init__(
            input_shape, total_food_category, total_ingredients_category, name
        )
        self.augmentation_layers = self.get_augmentation_layers()
        self.preprocess_layers = self.get_preprocess_layers(self.augmentation_layers)
        self.convolution_block = self.get_convolution_block()
        self.shared_layers = self.get_shared_layers(self.convolution_block)

        self.category_classification_layers = self.get_category_classification_layers(
            self.shared_layers, total_food_category
        )
        self.shared_nutrition_regression_layers = self.get_shared_nutrition_layers(
            self.shared_layers
        )
        self.calorie_regression_layers = self.get_calorie_regression_layers(
            self.shared_nutrition_regression_layers
        )
        self.carbs_regression_layers = self.get_carbs_regression_layers(
            self.shared_nutrition_regression_layers
        )
        self.protein_regression_layers = self.get_protein_regression_layers(
            self.shared_nutrition_regression_layers
        )
        self.fat_regression_layers = self.get_fat_regression_layers(
            self.shared_nutrition_regression_layers
        )
        self.ingredients_multilabel_layers = self.get_ingredients_multilabel_layers(
            self.convolution_block, total_ingredients_category
        )

    def get_augmentation_layers(self):
        input_layer = keras.layers.Input(shape=self.input_shape)
        augmentation_layer = keras.layers.RandomFlip()(input_layer)
        augmentation_layer = keras.layers.RandomRotation(0.2)(augmentation_layer)
        return keras.Model(
            inputs=input_layer, outputs=augmentation_layer, name="augmentation_layers"
        )

    def get_preprocess_layers(self, previous_layer):
        return None

    def get_convolution_block(self):
        return None

    def get_shared_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        shared_layer = keras.layers.Flatten()(input_layer)
        shared_layer = keras.layers.Dense(
            128, activation="relu", name="shared_dense_1"
        )(shared_layer)
        shared_layer = keras.layers.BatchNormalization()(shared_layer)
        output_layer = keras.layers.Dropout(0.2)(shared_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="shared_layers"
        )

    def get_category_classification_layers(self, previous_layer, total_categories):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        category_classification_layer = keras.layers.Dense(
            total_categories, activation="softmax", name="category_output"
        )(input_layer)
        return keras.Model(
            inputs=input_layer,
            outputs=category_classification_layer,
            name="category_output",
        )

    def get_shared_nutrition_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        nutrition_regression_layers = keras.layers.Dense(
            128, activation="relu", name="nutrition_dense_1"
        )(input_layer)
        nutrition_regression_layers = keras.layers.BatchNormalization()(
            nutrition_regression_layers
        )
        output_layer = keras.layers.Dropout(0.1)(nutrition_regression_layers)
        return keras.Model(
            inputs=input_layer,
            outputs=output_layer,
            name="nutrition_regression_shared_layers",
        )

    def get_calorie_regression_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        calorie_regression_layers = keras.layers.Dense(1, name="calorie_output")(
            input_layer
        )
        return keras.Model(
            inputs=input_layer, outputs=calorie_regression_layers, name="calorie_output"
        )

    def get_carbs_regression_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        carbs_regression_layers = keras.layers.Dense(1, name="carbs_output")(
            input_layer
        )
        return keras.Model(
            inputs=input_layer, outputs=carbs_regression_layers, name="carbs_output"
        )

    def get_protein_regression_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        protein_regression_layers = keras.layers.Dense(1, name="protein_output")(
            input_layer
        )
        return keras.Model(
            inputs=input_layer, outputs=protein_regression_layers, name="protein_output"
        )

    def get_fat_regression_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        fat_regression_layers = keras.layers.Dense(1, name="fat_output")(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=fat_regression_layers, name="fat_output"
        )

    def get_ingredients_multilabel_layers(self, previous_layer, total_ingredients):
        # get the shape of the feature map (batch_size,height,width,channels)
        feature_map_shape = previous_layer.output_shape[1:]
        feature_map_height = feature_map_shape[0]
        feature_map_width = feature_map_shape[1]

        input_layer = keras.layers.Input(feature_map_shape)

        # crop the feature map into grids of feature_map_height * feature_map_width
        region_branches = []
        for row in range(1, feature_map_height + 1):
            top_crop = row - 1
            bottom_crop = feature_map_height - row
            for col in range(1, feature_map_width + 1):
                left_crop = col - 1
                right_crop = feature_map_width - col
                crop_layer = keras.layers.Cropping2D(
                    cropping=((top_crop, bottom_crop), (left_crop, right_crop))
                )(input_layer)
                flatten_layer = keras.layers.Flatten()(crop_layer)
                ingredient_prediction = keras.layers.Dense(
                    total_ingredients, activation="softmax"
                )(flatten_layer)
                reshape_layer = keras.layers.Reshape((1, total_ingredients))(
                    ingredient_prediction
                )  # reshape for global pooling 1D
                region_branches.append(reshape_layer)
        concatenate_layer = keras.layers.Concatenate(axis=1)(region_branches)
        global_max_pooling_layer = keras.layers.GlobalMaxPool1D()(concatenate_layer)
        output_layer = keras.layers.Dense(total_ingredients, activation="sigmoid")(
            global_max_pooling_layer
        )
        return keras.Model(input_layer, output_layer, name="ingredients_output")

    def build_and_compile(
        self,
        category_classification_loss=keras.losses.CategoricalCrossentropy(),
        calorie_regression_loss=keras.losses.MeanAbsoluteError(),
        carbs_regression_loss=keras.losses.MeanAbsoluteError(),
        protein_regression_loss=keras.losses.MeanAbsoluteError(),
        fat_regression_loss=keras.losses.MeanAbsoluteError(),
        ingredient_multilabel_loss=keras.losses.BinaryCrossentropy(),
        category_classification_metrics=[
            keras.metrics.CategoricalAccuracy(),
            keras.metrics.Precision(),
            keras.metrics.Recall(),
        ],
        calorie_regression_metrics=[keras.metrics.MeanAbsoluteError()],
        carbs_regression_metrics=[keras.metrics.MeanAbsoluteError()],
        protein_regression_metrics=[keras.metrics.MeanAbsoluteError()],
        fat_regression_metrics=[keras.metrics.MeanAbsoluteError()],
        ingredient_multilabel_metrics=[
            keras.metrics.TopKCategoricalAccuracy(5),
            keras.metrics.Precision(),
            keras.metrics.Recall(),
        ],
        category_classification_loss_weights=1.0,
        ingredient_multilabel_loss_weights=1.0,
        calorie_regression_loss_weights=1.0,
        carbs_regression_loss_weights=1.0,
        protein_regression_loss_weights=1.0,
        fat_regression_loss_weights=1.0,
    ):
        assert (
            self.preprocess_layers != None
        ), "Error: No preprocess layers exists. Class RegionWiseModel should not be instantiated."
        assert (
            self.convolution_block != None
        ), "Error: No convolution block exists. Class RegionWiseModel should not be instantiated."
        model_inputs = self.input_layer
        x = self.augmentation_layers(model_inputs)
        x = self.preprocess_layers(x)
        x = self.convolution_block(x)
        ingredients_multilabel_head = self.ingredients_multilabel_layers(x)
        x = self.shared_layers(x)
        category_classification_head = self.category_classification_layers(x)
        nutrition_regression_head = self.shared_nutrition_regression_layers(x)
        calorie_regression_head = self.calorie_regression_layers(
            nutrition_regression_head
        )
        carbs_regression_head = self.carbs_regression_layers(nutrition_regression_head)
        protein_regression_head = self.protein_regression_layers(
            nutrition_regression_head
        )
        fat_regression_head = self.fat_regression_layers(nutrition_regression_head)

        model = keras.Model(
            inputs=model_inputs,
            outputs=[
                category_classification_head,
                ingredients_multilabel_head,
                calorie_regression_head,
                carbs_regression_head,
                protein_regression_head,
                fat_regression_head,
            ],
            name=self.name,
        )
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.0001),
            loss={
                "category_output": category_classification_loss,
                "calorie_output": calorie_regression_loss,
                "carbs_output": carbs_regression_loss,
                "protein_output": protein_regression_loss,
                "fat_output": fat_regression_loss,
                "ingredients_output": ingredient_multilabel_loss,
            },
            metrics={
                "category_output": category_classification_metrics,
                "calorie_output": calorie_regression_metrics,
                "carbs_output": carbs_regression_metrics,
                "protein_output": protein_regression_metrics,
                "fat_output": fat_regression_metrics,
                "ingredients_output": ingredient_multilabel_metrics,
            },
            loss_weights={
                "category_output": category_classification_loss_weights,
                "calorie_output": calorie_regression_loss_weights,
                "carbs_output": carbs_regression_loss_weights,
                "protein_output": protein_regression_loss_weights,
                "fat_output": fat_regression_loss_weights,
                "ingredients_output": ingredient_multilabel_loss_weights,
            },
        )
        self.model = model
        return model

In [ ]:
class RegionWiseMobileNetv2Model(RegionWiseModel):
    def __init__(self, input_shape, total_food_category, total_ingredients_category):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "RegionWiseFoodNet_with_MobileNetv2",
        )
        self.preprocess_layers = self.get_preprocess_layers(self.augmentation_layers)
        self.convolution_block = self.get_convolution_block()

    def get_preprocess_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        output_layer = keras.applications.mobilenet_v2.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        mobilenet_v2_convolution_layers = keras.applications.MobileNetV2(
            input_shape=self.input_shape, include_top=False, weights="imagenet"
        )
        mobilenet_v2_convolution_layers.trainable = False
        return mobilenet_v2_convolution_layers

In [ ]:
regionwise_foodnet_mobilenetv2 = RegionWiseMobileNetv2Model(
    (224, 224, 3),
    len(one_hot_encoder.all_food_categories),
    len(one_hot_encoder.all_ingredients),
)
regionwise_foodnet_mobilenetv2.build_and_compile()
regionwise_foodnet_mobilenetv2.model.summary()

In [ ]:
keras.utils.plot_model(
    regionwise_foodnet_mobilenetv2.model,
    "./models/regionwise_foodnet_mobilenetv2.png",
    show_shapes=True,
    expand_nested=False,
)

In [ ]:
regionwise_foodnet_mobilenetv2.model.fit(
    training_dataset, epochs=1, verbose=1, validation_data=validation_dataset
)

## Testing 

In [ ]:
def load_recipe5k_metadata():
    directory = (
        Path("../Food Datasets/final-dataset") / "metadata" / "recipes5k_metadata.csv"
    )
    return pd.read_csv(directory, sep="\t")

In [ ]:
data = load_recipe5k_metadata()

In [ ]:
data.head()

In [ ]:
test_recipes5k = Recipes5k()

In [ ]:
len(test_recipes5k)

In [ ]:
test_gen_func = test_recipes5k.generate_dataset()

In [ ]:
test_dataset = test_recipes5k.get_dataset()

In [ ]:
list(recipes5k.take(1))

In [ ]:
test_model = Model()

In [ ]:
test_model = test_model.build_and_compile()

In [ ]:
test_model.summary()

In [ ]:
test_model.fit(
    x=test_recipes5k.training_dataset,
    epochs=1,
    verbose=1,
    validation_data=test_recipes5k.validation_dataset,
)

In [ ]:
test = tf.constant([[[1, 2, 3]]])

In [ ]:
test.shape

In [ ]:
tf.expand_dims(test, axis=0)

In [ ]:
test_recipes5k.training_dataset

In [ ]:
test_recipes5k.validation_dataset

In [ ]:
row = 0
for x in test_recipes5k.training_dataset:
    row += 1
print(row)

In [ ]:
# check training dataset (first 70%)
training_rows = int(len(recipes5k) * 0.7)
training_data = recipes5k.metadata.iloc[:training_rows]
validation_data = recipes5k.metadata.iloc[training_rows:,]

In [ ]:
from pprint import pprint

In [ ]:
# check categories of training
category_count = {x: 0 for x in recipes5k.extract_all_categories()}
for x in training_data["Category"].tolist():
    category_count[x] += 1

In [ ]:
pprint(category_count)

In [ ]:
validation_count = {x: 0 for x in recipes5k.extract_all_categories()}
for x in validation_data["Category"].tolist():
    validation_count[x] += 1

In [ ]:
pprint(validation_count)

In [ ]:
list(recipes5k.training_dataset.take(1))

In [ ]:
mobilenet_v2_convolution_layers = keras.applications.MobileNetV2(
    input_shape=(224, 224, 3), include_top=False, weights="imagenet"
)

In [ ]:
mobilenet_v2_convolution_layers.layers[-1].output_shape[1:3]

In [ ]:
mobilenet_v2_convolution_layers.output_shape

In [ ]:
test_model = RegionWise_FoodNet_MobileNetv2()

In [ ]:
final_model = test_model.mobilenetv2_convolution_block(test_model.input_layer)

In [ ]:
final_model = test_model.ingredient_classifier(final_model)

In [ ]:
t = keras.Model(test_model.input_layer, final_model)

In [ ]:
t.summary()

In [ ]:
test_model.ingredient_classifier.summary()

In [ ]:
import numpy as np

x = keras.layers.concatenate(
    [
        tf.constant([1, 0, 1]),
        tf.constant([0, 0, 2]),
        tf.constant([1, 0, 1]),
        tf.constant([5, 0, 1]),
    ],
    axis=0,
)

In [ ]:
s = tf.reshape(x, (1, 4, 3))

In [ ]:
keras.layers.MaxPooling1D()(s)

In [ ]:
keras.layers.GlobalMaxPooling1D()(s)

In [ ]:
df = tf.data.Dataset.range(300000)
df = df.shuffle(200000)
list(df.take(1))

In [ ]:
#     # Overriding the method from DatasetLoader
#     def generate_dataset(self):
#         img_dir = self.dir_path / "images"
#         for index, row in self.metadata.iterrows():
#             dish_dir = img_dir / row["Category"] / row["dish_id"]
#             for img_path in dish_dir.iterdir():
#                 assert (
#                     img_path.suffix == ".jpeg" or img_path.suffix == ".png"
#                 ), f"{img_path} is not an expected image file"
#                 img_tensor = self.load_image_to_arr(img_path)
#                 calorie_tensor = row["Calorie(kcal)"]
#                 carbs_tensor = row["Carbohydrate(g)"]
#                 protein_tensor = row["Protein(g)"]
#                 fat_tensor = row["Fat(g)"]
#                 one_hot_category_tensor = (
#                     self.one_hot_encoder.get_category_one_hot_encoding(row["Category"])
#                 )
#                 one_hot_ingredient_tensor = (
#                     self.one_hot_encoder.get_ingredients_one_hot_encoding(
#                         row["Ingredients"].split(",")
#                     )
#                 )
#                 yield tf.constant(img_tensor), {
#                     "category_output": tf.constant(one_hot_category_tensor),
#                     "calorie_output": tf.constant(calorie_tensor),
#                     "carbs_output": tf.constant(carbs_tensor),
#                     "protein_output": tf.constant(protein_tensor),
#                     "fat_output": tf.constant(fat_tensor),
#                     "ingredients_output": one_hot_ingredient_tensor,
#                 }

#     # Overriding the method from DatasetLoader
#     def get_dataset(self):
#         dataset = tf.data.Dataset.from_generator(
#             self.generate_dataset,
#             output_signature=(
#                 tf.TensorSpec(shape=(224, 224, 3), dtype=tf.dtypes.float32),
#                 {
#                     "category_output": tf.TensorSpec(
#                         shape=(len(self.one_hot_encoder.all_food_categories)),
#                         dtype=tf.dtypes.float32,
#                     ),
#                     "calorie_output": tf.TensorSpec(shape=(), dtype=tf.dtypes.float32),
#                     "carbs_output": tf.TensorSpec(shape=(), dtype=tf.dtypes.float32),
#                     "protein_output": tf.TensorSpec(shape=(), dtype=tf.dtypes.float32),
#                     "fat_output": tf.TensorSpec(shape=(), dtype=tf.dtypes.float32),
#                     "ingredients_output": tf.TensorSpec(
#                         shape=(len(self.one_hot_encoder.all_ingredients)),
#                         dtype=tf.dtypes.float32,
#                     ),
#                 },
#             ),
#         )
#         return dataset.shuffle(len(self.metadata) * 30, seed=1234)

In [ ]:
tf.constant("sting")